# metric&loss for segmentation


In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras.backend as K

import tensorflow.keras.losses as losses

import random
import numpy as np

In [ ]:
sample1 = tf.constant([[0, 1, 0],
                       [1, 1, 1],
                       [0, 0, 0]], dtype=tf.float32)

sample2 = tf.constant([[0, 0, 0],
                       [1, 1, 1],
                       [0, 1, 0]], dtype=tf.float32)

## metric
1.  IoU
2. Dice coef

#### IoU

In [ ]:
def iou(y_true, y_pred, dtype=tf.float32):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)

    area_true = tf.reduce_sum(y_true_f)
    area_pred = tf.reduce_sum(y_pred_f)
    union = area_true + area_pred - intersection

    iou = tf.where(tf.equal(union, 0), 0.0, tf.math.divide(intersection, union))
    return iou.numpy()

In [ ]:
iou(sample1, sample2)

0.6

#### dice coef

In [ ]:
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return dice.numpy()

In [ ]:
dice_coef(sample1, sample2)

0.75

## Loss

1. iou loss
2. dice loss
3. dice BCE loss
4. focal loss
5. Tversky loss

### iou loss

In [ ]:
def iou_loss(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)

    area_true = tf.reduce_sum(y_true_f)
    area_pred = tf.reduce_sum(y_pred_f)
    union = area_true + area_pred - intersection

    iou = tf.where(tf.equal(union, 0), 0.0, tf.math.divide(intersection, union))

    return 1 - iou.numpy()

In [ ]:
iou_loss(sample1, sample2)

0.3999999761581421

#### dice loss
$$diceloss = 1 - dicecoef$$

In [ ]:
def dice_loss(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1 - dice

#### dice BCE loss

$$ dice BCE loss = BCE + diceloss$$

In [ ]:
def dice_BCE_loss(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    dice_loss = 1 - (2*intersection + smooth) / (K.sum(y_true) + K.sum(y_pred) + smooth)

    BCE =  losses.binary_crossentropy(y_true_f, y_pred_f)

    Dice_BCE_loss = BCE + dice_loss

    return Dice_BCE_loss.numpy()

In [ ]:
dice_BCE_loss(sample1, sample2)

3.6675763

#### focal loss
- 2017년, 극도로 불균형한 데이터 세트를 해결하기 위한 수단으로 도입

$$ Focal loss = BCE*(1-pred)^r$$

In [ ]:
alpha = 0.8
gamma = 2

def focal_loss(targets, inputs, alpha=alpha, gamma=gamma):
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)

    bce = K.binary_crossentropy(targets, inputs)
    bce_exp = K.exp(-bce)
    focal_loss = K.mean(alpha * K.pow((1-bce_exp), gamma) * bce)

    return focal_loss

#### Tversky Loss
- 서로 다른 유형의 오류가 얼마나 심하게 처벌되는지를 조정할 수 있는 상수를 활용하여 불균형 데이터셋에 대해 최적화하도록 설계

$$ 1 - TP/(TP + a*FP + b*FN) $$

In [ ]:
alpha = 0.5
beta = 0.5

def tversky_loss(targets, inputs, alpha=alpha, beta=beta, smooth=1e-6):
        inputs = K.flatten(inputs)
        targets = K.flatten(targets)

        tp = K.sum((inputs * targets))
        fp = K.sum(((1-targets) * inputs))
        fn = K.sum((targets * (1-inputs)))

        tversky = (tp + smooth)/(tp + alpha*fp + beta*fn + smooth)

        return 1 - tversky

출처:  
https://github.com/yingkaisha/keras-unet-collection/tree/d30f14a259656d2f26ea11ed978255d6a7d0ce37

https://www.kaggle.com/code/bigironsphere/loss-function-library-keras-pytorch/notebook
  
https://github.com/zhengyang-wang/3D-Unet--Tensorflow/blob/c5d603a69243a69dd6d89edefdf1ba249640450b/utils/HausdorffDistance.py#L8